## 导入数据

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
from copy import deepcopy
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch.utils.data.sampler import RandomSampler
import matplotlib.pyplot as plt
import cv2

## 配置

In [2]:
# 需要修改成的数据大小
dsize = 128
# 最大学习率(优化器)
max_lr = 0.012 
# 正则项权值的衰减(优化器)
weight_decay = 1e-4 
# 一般0.9 (优化器)
momentum = 0.9 
# 最小学习率(退火学习)
min_lr = 0.001
# 设置GPU运行
device = torch.device('cuda')
# 退火学习，下降次数
scheduler_step = 300
# 打包个数
batch_size = 5

## 文件路径

In [3]:
src = './competition_data'
save_weight_path = src + '/weight'
train_image_dir = src + '/train/images'
train_mask_dir = src + '/train/masks'
test_image_dir = src + '/test/images'

# 获取数据id

In [4]:
depths = pd.read_csv(src + '/train.csv')
fold = (list(range(5))*1000)[:len(depths)] # [0,1,2,3,4,0,1,2...]
depths['fold'] = fold # 将数据标记为五份
all_ids = depths['id'].values # 取出所有id


## 图片id分为五类

In [5]:
fold = []
for i in range(5):
  tem = depths.loc[depths['fold']==i,'id'].values
  fold.append(tem)

## 获取图片（输入，输出）

In [6]:
def get_train_images(ids):
  images = []
  masks = []
  for id in ids:
    image = plt.imread(train_image_dir+'/'+id+'.png')[0] / 255
    mask = plt.imread(train_mask_dir+'/'+id+'.png')[0] / 255
    masks.append(mask)
    images.append(image)
  return images,masks

## 创建数据集类型
数据集类型有三个常用魔法方法
1. 初始化（获取参数）
2. 获取数据（数据处理，返回数据）
3. 获取数据集长度（返回数据集长度）

In [7]:
# 训练数据集
class TensorDataset(Dataset):
  def __init__(self, data, target):
    self.data = data
    self.target = target

  def __getitem__(self, index):
    # 改变尺寸，并且变为张量
    resolved_data = torch.Tensor(
      cv2.resize(self.data[index], dsize=(dsize,dsize))
    ).reshape(1,dsize,dsize)
    # 改变尺寸，并且变为张量
    resolved_target = torch.Tensor(
      cv2.resize(self.target[index], dsize=(dsize,dsize))
    ).reshape(1,dsize,dsize)
    # 返回
    # (1,128,128),(1,128,128)
    return resolved_data,resolved_target

  def __len__(self):
    return len(self.data)

## 我的模型

In [8]:
class Decoder(nn.Module):
  def __init__(self,in_size,out_size,mid_size) -> None:
    super().__init__()
    self.layer1 = nn.ConvTranspose2d(in_channels=in_size,out_channels=out_size,kernel_size=2,stride=2,padding=0)
    self.layer2 = nn.Sequential(
      nn.Conv2d(in_channels=mid_size,out_channels=out_size,stride=1,padding=1,kernel_size=3),
      nn.ReLU()
    )
  def forward(self,x1,x2):
    y1 = self.layer1(x1)
    tem = torch.cat((y1,x2),dim=1)
    y2 = self.layer2(tem)
    return y2

class SimpleUNet(nn.Module):
  def __init__(self) -> None:
    super().__init__()
    self.layer1 = nn.Sequential(
      nn.Conv2d(in_channels=1,out_channels=8,stride=2,padding=1,kernel_size=3),
      nn.BatchNorm2d(num_features=8),
      nn.ReLU()
    )
    self.layer2 = nn.Sequential(
      nn.Conv2d(in_channels=8,out_channels=64,stride=2,padding=1,kernel_size=3),
      nn.BatchNorm2d(num_features=64),
      nn.ReLU()
    )
    self.layer3 = nn.Sequential(
      nn.Conv2d(in_channels=64,out_channels=512,stride=2,padding=1,kernel_size=3),
      nn.BatchNorm2d(num_features=512),
      nn.ReLU()
    )
    self.layer4 = nn.Sequential(
      nn.Conv2d(in_channels=512,out_channels=1024,stride=2,padding=1,kernel_size=3),
      nn.BatchNorm2d(num_features=1024),
      nn.ReLU()
    )
    self.decoder1 = Decoder(1024,512,512+512)
    self.decoder2 = Decoder(512,256,256+64)
    self.decoder3 = Decoder(256,128,128+8)
    self.last = nn.Sequential(
      nn.UpsamplingBilinear2d(scale_factor=2),
      nn.Conv2d(in_channels=128,kernel_size=5,out_channels=64,padding=2),
      nn.Conv2d(in_channels=64,kernel_size=1,out_channels=1)
    )
    


  def forward(self,input):
    x1 = self.layer1(input) # torch.Size([18, 8, 64, 64])
    x2 = self.layer2(x1) # torch.Size([18, 64, 32, 32])
    x3 = self.layer3(x2) # torch.Size([18, 512, 16, 16])
    x4 = self.layer4(x3) # torch.Size([18, 1024, 8, 8])

    y1 = self.decoder1(x4,x3) # torch.Size([18, 512, 16, 16])
    y2 = self.decoder2(y1,x2)# torch.Size([18, 256, 32, 32])
    y3 = self.decoder3(y2,x1) # torch.Size([18, 128, 64, 64])
    output = self.last(y3) # torch.Size([18, 1, 128, 128])

    return output


In [9]:
# inp = torch.Tensor(12,1,128,128)
# models = SimpleUNet()
# outp = models(inp)
# outp.size()

## 获取模型

In [10]:
def get_model(key):
  model = SimpleUNet()
  model.load_state_dict(torch.load("./weight/" + key+ ".pth"))
  return model

## 创建模型对象

In [11]:
salt = SimpleUNet()
# salt = get_model("0_4")

# GPU 运算
salt.to(device=device)

SimpleUNet(
  (layer1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(8, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer3): Sequential(
    (0): Conv2d(64, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer4): Sequential(
    (0): Conv2d(512, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (decoder1): Decoder(
    (layer1): ConvTranspose2d(1024, 512, kernel_size=(2, 2), stride=(2, 2))
    (layer2): Sequential(
      (0): Conv2d(1024, 512,

## 进行一次训练

In [12]:

optimizer = torch.optim.SGD(salt.parameters(), lr=max_lr, momentum=momentum, weight_decay=weight_decay)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, scheduler_step, min_lr)

def train(loader_data,model):
  running_loss = 0
  model.train()
  for input,mask in loader_data:
    input, mask = input.to(device), mask.to(device)
    
    optimizer.zero_grad()# 梯度初始化为零
    # 使用with，会自动关闭梯度计算
    # 设置梯度可算
    with torch.set_grad_enabled(True):
      logit = model(input)# 进行一次计算
      loss = nn.BCEWithLogitsLoss()(logit.squeeze(),mask.squeeze())# 计算误差
      loss.backward()# 反馈
      optimizer.step()# 进行一次参数更新
    running_loss += loss.item()*input.size()[0]# 累计平均误差
  epoch_loss = running_loss / len(loader_data)# 计算平均误差
  return epoch_loss

## 进行一次测试

In [13]:
def test(loader_test,model):
  running_loss = 0.0
  data_size = len(loader_test)
  # 测试
  model.eval()
  for input, mask in loader_test:
    input, mask = input.to(device), mask.to(device)
    with torch.set_grad_enabled(False):
      output = model(input)
      loss = nn.BCEWithLogitsLoss()(output.squeeze(), mask.squeeze())
    running_loss += loss.item() * input.size(0)
  return running_loss/data_size

## 主函数部分

In [14]:
for idx in range(5):
  if idx == 1:
    break

  optimizer = torch.optim.SGD(salt.parameters(), lr=max_lr, momentum=momentum, weight_decay=weight_decay)
  lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, scheduler_step, min_lr)

  # setdiff1d 取不同的元素
  train_id = np.setdiff1d(all_ids, fold[idx])
  val_id = fold[idx]
  # 取出数据
  X_train, y_train = get_train_images(train_id)
  X_val, y_val = get_train_images(val_id)
  # 制作数据集
  train_data = TensorDataset(X_train, y_train)
  val_data = TensorDataset(X_val, y_val)
  # 打乱，制作可迭代数据集
  train_loader = DataLoader(train_data,shuffle=True,batch_size=batch_size) 
  val_loader = DataLoader(val_data,shuffle=False,batch_size=batch_size) 

  num_snapshot = 0
  lowest_loss = 10000
# 训练
  for epoch_ in range(300): # 300
    train_loss = train(train_loader, salt)
    val_loss = test(val_loader, salt)
    # 每训练一次调整学习率（退火学习）
    lr_scheduler.step()
    if epoch_ % 5 == 0:
      torch.save(salt.state_dict(), "./weight/"+ "epoch_" + str(epoch_) + '.pth')

    # if lowest_loss > val_loss:
    #   lowest_loss = val_loss
    #   best_param = salt.state_dict()

    # 调节一个
    if (epoch_ + 1) % scheduler_step == 0:
      # torch.save(best_param, "./weight/" + str(idx) +"_"+ str(num_snapshot) + '.pth')
      # 重置优化器，以及退火学习
      optimizer = torch.optim.SGD(salt.parameters(), lr=max_lr, momentum=momentum, weight_decay=weight_decay)
      lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, scheduler_step, min_lr)
      num_snapshot += 1
      lowest_loss = 10000

    print('epoch: {} train_loss: {:.4f} val_loss: {:.4f}  lr: {:.4f}'.format(epoch_ + 1, train_loss*100, val_loss*100, lr_scheduler.get_last_lr()[0]))

epoch: 1 train_loss: 11.8775 val_loss: 3.5958  lr: 0.0120
epoch: 2 train_loss: 3.7043 val_loss: 3.5617  lr: 0.0120
epoch: 3 train_loss: 3.6766 val_loss: 3.5369  lr: 0.0120
epoch: 4 train_loss: 3.6577 val_loss: 3.5233  lr: 0.0120
epoch: 5 train_loss: 3.6446 val_loss: 3.5182  lr: 0.0120
epoch: 6 train_loss: 3.6346 val_loss: 3.5066  lr: 0.0120
epoch: 7 train_loss: 3.6258 val_loss: 3.5054  lr: 0.0120
epoch: 8 train_loss: 3.6196 val_loss: 3.4944  lr: 0.0120
epoch: 9 train_loss: 3.6162 val_loss: 3.4940  lr: 0.0120
epoch: 10 train_loss: 3.6134 val_loss: 3.4861  lr: 0.0120
epoch: 11 train_loss: 3.6056 val_loss: 3.4809  lr: 0.0120
epoch: 12 train_loss: 3.6041 val_loss: 3.4803  lr: 0.0120
epoch: 13 train_loss: 3.6019 val_loss: 3.4797  lr: 0.0119
epoch: 14 train_loss: 3.5972 val_loss: 3.4756  lr: 0.0119
epoch: 15 train_loss: 3.5965 val_loss: 3.4713  lr: 0.0119
epoch: 16 train_loss: 3.5950 val_loss: 3.4782  lr: 0.0119
epoch: 17 train_loss: 3.5933 val_loss: 3.4664  lr: 0.0119
epoch: 18 train_loss: 

KeyboardInterrupt: 

## torch 保存参数

|操作|函数|
|-|-|
|保存|torch.save(model.state_dict(),path)|
|读取|torch.load(path)|